# Cours de Statistique: Chapitre 4

Intervalles de confiance et tests

In [1]:
# Uncomment the following and run if not installed.
# %pip install -q ipywidgets

In [40]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm
from scipy.special import ndtri

from ipywidgets import interactive, fixed, widgets

## Intervalles de confiance

### Code

In [3]:
def distribution_bernoulli(x, theta):
    if x == 0:
        return 1 - theta
    if x == 1:
        return theta
    return 0

def distribution_gaussienne(x: float, moyenne=0., variance=1.):
    return np.exp(-(x - moyenne)**2 / (2 * variance)) / (np.sqrt(2 * np.pi * variance))

def distribution_poisson(k, theta=1.):
    assert theta > 0
    if k < 0:
        return 0
    k = int(k)
    return np.exp(-theta) * theta**k / np.math.factorial(k)

def distribution_uniforme(x: float, theta=1.):
    if np.abs(theta) < 1e-14:
        return 0. if np.abs(x) > 0 else 1.
    return (1 if 0 <= x <= theta else 0) / (theta)

def distribution_exponentielle(x, theta=1.):
    assert theta > 0
    if x < 0:
        return 0
    return theta * np.exp(-theta * x)

In [18]:
def plot_gaussienne(moyenne, variance=1., show_moyenne=True, show_variance=True, style="b-"):
    X = np.linspace(-2., 2., 1000)
    plt.plot(X, distribution_gaussienne(X, moyenne, variance), style)
    if show_moyenne:
        plt.annotate(text=r"$\mu$",  ha = "center", va="bottom", xy=(moyenne, distribution_gaussienne(moyenne, moyenne, variance)), xytext=(moyenne, 0.), arrowprops={'arrowstyle': '-', 'color': 'green', 'ls': '-'}, color = 'green')
    if show_variance:
        plt.annotate(text="", xy=(moyenne - variance, distribution_gaussienne(moyenne - variance, moyenne, variance)), xytext=(moyenne + variance, distribution_gaussienne(moyenne + variance, moyenne, variance)), arrowprops={'arrowstyle': '<->', 'color': 'red', 'ls': '--'})
        plt.text(moyenne, distribution_gaussienne(moyenne+variance, moyenne, variance), r"$\sigma^2$", ha="center", va="center", color='red', bbox={'boxstyle': 'square', 'fc':'w','ec': 'w'})
    plt.ylim(bottom=0)

In [20]:
TRUE_MU = 0
TRUE_SIGMA = 1.
observations = np.random.normal(TRUE_MU, TRUE_SIGMA, size=1000)

In [48]:
def exemple_intervalle_confiance_gaussienne(
    observations,
    n=10,
    erreur=0.05,
    show_moyenne=True,
    show_moyenne_empirique=True,
):
    n = min(n, len(observations))
    moyenne_empirique = observations[:n].mean()
    # eps = norm.ppf((2-alpha)/2)/np.sqrt(n)
    eps = ndtri(1-erreur/2)/np.sqrt(n)
    plot_gaussienne(0, 1, show_moyenne, show_variance=False)
    plt.annotate(text=r'-ε', ha="center", va="bottom", xy=(moyenne_empirique-eps, distribution_gaussienne(moyenne_empirique-eps, 0, 1)), xytext=(moyenne_empirique-eps, 0.), arrowprops={'arrowstyle': '-', 'color': 'red', 'ls': '--'}, color = 'red')
    plt.annotate(text=r'+ε', ha="center", va="bottom", xy=(moyenne_empirique+eps, distribution_gaussienne(moyenne_empirique+eps, 0, 1)), xytext=(moyenne_empirique+eps, 0.), arrowprops={'arrowstyle': '-', 'color': 'red', 'ls': '--'}, color = 'red')
    if show_moyenne_empirique:
        moyenne_empirique = np.mean(observations[:n])
        plt.annotate(text=r"$\hat{\mu}}$",  ha = "center", va="bottom", xy=(moyenne_empirique, distribution_gaussienne(moyenne_empirique, 0, 1)), xytext=(moyenne_empirique, 0.), arrowprops={'arrowstyle': '-', 'color': 'red', 'ls': '--'}, color = 'red')
    X = np.linspace(max(-2, moyenne_empirique - eps), min(moyenne_empirique + eps, 2), 1000)
    Y = distribution_gaussienne(X, 0, 1)
    plt.fill_between(X, Y, 0, color='red', alpha=.1)




### Résultats

In [49]:
interactive(exemple_intervalle_confiance_gaussienne,
            observations = fixed(observations),
            n=widgets.IntSlider(min=1, max=100, step=1, value=10),
            erreur=widgets.FloatSlider(min=0, max=0.1, step=0.01, value=0.05),
)

interactive(children=(IntSlider(value=10, description='n', min=1), FloatSlider(value=0.05, description='erreur…